<a href="https://colab.research.google.com/github/jlee2843/Peer-Review-22-23/blob/Albert-2022/API_Biorvix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print("version:", sys.version)

#updates outdate packages
#pip list --outdated --format=freeze | awk -F '==' '{print $1}' | xargs -n1 pip install -U

version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]


In [3]:
#installing of models
!pip3 install --upgrade pip
!pip3 install python-doi
!pip3 install --upgrade pyarrow
!pip3 install multipledispatch


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 10.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

**Biorxiv API** information is found [here](https://api.biorxiv.org/details/medrxiv/help).<br>
**Final edit:** December 02, 2022.<br>
**NB:** Using Jenny's notebook  as a template
<br>
tqdm is used as visualization for the processing fo data. (Its manual can be found https://github.com/tqdm/tqdm#manual)
<br>
<br>
**NB: Some issues when using BioRxiv API**
1. there are empty entry e.g. no subject area declaired
    - sometimes these entries can be fixed and sometimes it cannot. For example, one can manual enter the corresponding other but there is no way to find the intended subject area when it is not given.
2. sometimes the publication date is before the preprint date
3. sometimes the date return is not in the format of 'yyyy-mm-dd' when querying for published article detail
4. sometimes the information returned have miscellanous information for example '- Genetic Institute...' in the author field which requires manual reentry of information

# Common functions

In [17]:
%reset
import sys
import os
print(sys.path)
import utils
print(utils.__file__)


Once deleted, variables cannot be recovered. Proceed (y/[n])?  yes


['/tmp/albert.leung/sparklocaldirs/spark-4777478f-ff76-44ed-a60e-f83621343282/userFiles-e5a1717c-0f2f-46f2-9179-66f6ecb3c45b', '/home/albert.leung/proj/review/Peer-Review-22-23', '/home/albert.leung/proj/review/Peer-Review-22-23', '/global/software/spark/spark-3.2.0-bin-hadoop3.2/python', '/global/software/spark/sparkhpc/lib/python3.9/site-packages/sparkhpc-0.3.post4-py3.9.egg', '/global/software/spark/Anaconda3-2021.11/lib/python39.zip', '/global/software/spark/Anaconda3-2021.11/lib/python3.9', '/global/software/spark/Anaconda3-2021.11/lib/python3.9/lib-dynload', '', '/home/albert.leung/.local/lib/python3.9/site-packages', '/global/software/spark/Anaconda3-2021.11/lib/python3.9/site-packages', '/global/software/spark/Anaconda3-2021.11/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '$HOME/proj/review/Peer-Review-22-23/module', './module', '/home/albert.leung/proj/review/Peer-Review-22-23/module', '/home/albert.leung/proj/review/Peer-Review-22-23/module']
/home/albert.leung/proj/r

In [2]:
#from tqdm.notebook import tqdm, trange
import sys, os

sys.path.insert(0, f'{os.path.expanduser("~")}/proj/review/Peer-Review-22-23/utils.zip')

In [5]:
print(sys.path)

['/home/albert.leung/proj/review/Peer-Review-22-23/utils.zip', '/tmp/albert.leung/sparklocaldirs/spark-1afd3bb1-d236-4257-9796-a27636e4ca9b/userFiles-dc1b8bb7-5a62-4661-b629-6e7f381ec7f9', '/home/albert.leung/proj/review/Peer-Review-22-23', '/home/albert.leung/proj/review/Peer-Review-22-23', '/global/software/spark/spark-3.2.0-bin-hadoop3.2/python', '/global/software/spark/sparkhpc/lib/python3.9/site-packages/sparkhpc-0.3.post4-py3.9.egg', '/global/software/spark/Anaconda3-2021.11/lib/python39.zip', '/global/software/spark/Anaconda3-2021.11/lib/python3.9', '/global/software/spark/Anaconda3-2021.11/lib/python3.9/lib-dynload', '', '/home/albert.leung/.local/lib/python3.9/site-packages', '/global/software/spark/Anaconda3-2021.11/lib/python3.9/site-packages', '/global/software/spark/Anaconda3-2021.11/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg']


In [1]:

from typing import Union, List, Iterator
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from threading import current_thread
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
#from module.utils import setBasePathName, getBasePathName, setPath, getPath, setFileName, getFileName, checkDOI, \
#                         getSparkSqlContext, getSlurmID, getSlurmDir
from utils import *

import math
import pyspark.sql as sql
import pyspark.pandas as pd
import pyspark.rdd
import time
import numpy as np
import requests
import json
import urllib, urllib.request
import pathlib
import tqdm.contrib.concurrent as tq
import os
import atexit
import sys
import re

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# common helper function
def get_total (url) -> int:
    _, json_info = query.get_json_data(0, 0, url)
    return json_info["messages"][0]["total"]

def get_data(url:str, loop_range:range, disable:bool):
    #print(f"values: {list(loop_range)}")
    results = []
    #print(args)
    results = tq.thread_map(get_json_data, (f'{url}/{cursor}' for cursor in loop_range), 
                            desc=f'get_json_data {current_thread().name}', total=len(loop_range),
                            disable=disable, leave=False)
    
    return results


#def get_article_detail(url, articles):
#    result = get_data(url,articles)
#
#    return result

# Convert function to udf
'''
def ext_module(fx, parm):
    from module.utls import upperCaseUDF, titleCaseUDF, jsonDataUDF
    if fx == 'json':
        fx = get_json_data
    elif fx == 'upper':
        fx = upperCaseUDF
    else:
        fx = titleCaseUDF
    return fx(param)
'''

# prepublish helper function
def create_prepublish_df(df: sql.DataFrame) -> sql.DataFrame:
    from pyspark.sql.functions import date_format, trim, col
    
    try:
        #df.withColumn("Num_of_Authors", df.
        #df['Num_of_Authors'] = df.Authors.apply(lambda x: len(x.split(';')))
        df = df.withColumn("DOI",trim(col("DOI")))
        df = df.withColumn("Title", trim(col("Title"))).withColumn("Authors", trim(col("Authors")))
        df = df.withColumn("Corresponding_Authors", trim(col("Corresponding_Authors")))
        df = df.withColumn("Institution", arc_pyspark.upperCaseUDF(col("Institution")))
        df = df.withColumn("Date", date_format(col("Date"), "MM-dd-yyyy"))
        df = df.withColumn("Type", arc_pyspark.titleCaseUDF(col("Type"))).withColumn("Category", arc_pyspark.titleCaseUDF(col("Category")))
#        df.Xml = df.Xml.astype('str')
        df = df.withColumn("Published", trim(col("Published")))
    except Exception as e:
        print(f'Error in data format:{e.args}\n')
        print(e.with_traceback)

    return df

# helper functions for published articles
# pub = ["preprint_doi", "published_doi", "preprint_title", "preprint_authors", "preprint_author_corresponding", "preprint_author_corresponding_institution", "preprint_category", "published_journal", "preprint_date", "published_date"]
def create_published_df(df:sql.DataFrame) ->sql.DataFrame:
    try:
        df['Num_of_Authors'] = df.Authors.apply(lambda x: len(x.split(';')))
        df.DOI = df.DOI.astype('str')
        df.pub_DOI = df.pub_DOI.astype('str')
        df.Title = df.Title.astype('str').map(lambda x: x.strip())
        df.Authors = df.Authors.astype('str').map(lambda x: x.replace('\'','').strip())
        df.Corresponding_Authors = df.Corresponding_Authors.astype('str').str.strip()
        df.Institution = df.Institution.map(lambda x: x.strip().upper()).astype('category')
        df.Category = df.Category.map(lambda x: x.strip().title()).astype('category')
        df.Journal = df.Journal.astype('str').map(lambda x: x.strip().title()).astype('category')
        #tqdm.pandas(desc="Preprint_Date conversion!")
        df.Preprint_Date = df.Preprint_Date.map(lambda x: convert_date(x)).astype('datetime64[ns]')
        #tqdm.pandas(desc="Published_Date conversion!")
        df.Published_Date = df.Published_Date.map(lambda x: convert_date(x)).astype('datetime64[ns]')
    except Exception as e:
        print(f'Error in data format:{e.args}\n')
        print(e.with_traceback)
    #    print(f'Preprint_Date: {df.Preprint_Date} Published_Date: {df.Published_Date}')
#
    return df

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
        #df.Published_Date = datetime.strptime(df.Published_Date.str.strip().str.split(':')[0], '%Y-%m-%d')

'''
def multithread_processor(sc:pyspark.SparkContext, path:str, url:str, json_schema, col_names:List[str], step:int, loop_range:range, disable:bool):
    #print(f"values: {list(loop_range)}")
    columns = ["cursor", "url"]
    data = [(cursor, f'{url}/{cursor}') for cursor in loop_range]
    request_df = sqlCtx.createDataFrame(data=data, schema=columns)
    request_df.show()
    udf_executeRestApi = udf(f=get_json_data, returnType=json_schema)
    result_df = request_df.withColumn("result", udf_executeRestApi(pyspark.sql.functions.lit(0), col("url")))
    result_df.show()
    df = result_df.select(explode(col("result.collection")).alias("results"))
    df.show()
                                      
    #args = sc.parallelize()
    #print(args.take(5))
    #results = args.map(get_json_data)
    #print(args.take(1))
    #print(results.take(5))
    #tq.thread_map(lambda p: get_big_data(*p), args, desc='get_big_data', total=len(args))
    #df = query_to_df(sc, result, json_keys, col_names, range(cursor, cursor + step, step), disable)
    #df.saveAsPickleFile(pathlib.Path(f'{path}/{datetime.utcnow().timestamp()}.parquet'))
'''
'''
def get_json_data (counter:int, cursor:int,  url: str) -> List:
    try:
        request_API = requests.get(url)
        #print(f"{url} request_API: {request_API}")
        return cursor, json.loads(request_API.text)
    except Exception as e:
        if counter == 10 :
            raise e
            
        time.sleep(300)
        return cursor, get_json_data(counter + 1, cursor, url)
'''

def spark_rdd_querying_processing(sc: SparkContext, sqlCtx: SQLContext, path: str, url: str, schema: StructType, section: str,
                                  keys: List[str], colNames: List[str], loop_range:range):
    #sc.addPyFile(f'{os.path.expanduser("~")}/proj/review/Peer-Review-22-23/module/utils.py')
    #fx = get_json_data
    args = sc.parallelize([(0, cursor, f'{url}/{cursor}') for cursor in loop_range])
    #results = args.map(lambda p: ext_module('json', p))
    results = args.map(lambda p: query.get_json_data(*p))
    #print(f"get_json_data:\n {results.take(1)}")
    results = results.map(lambda p: (p[1], section, keys, p[0]))
    #print(f"args:\n {results.take(1)}")
    results = results.flatMap(lambda p: query.process_data(*p))
    df = sqlCtx.createDataFrame(data=results, schema=schema)
    df = df.toDF(*colNames)
    df = create_prepublish_df(df)
    #df.show()
    print(f'1: {path}')
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    filename = file.getFileName(path)
    df.write.option("header",True).partitionBy("Published").parquet(path=filename)
    print(f'filename: {filename}')
    #df.write.option("header",True).parquet(path=filename)

def multithread_processor(path:str, url:str, section: str, json_keys:List[str], col_names:List[str], step:int, loop_range:range, disable:bool):
    #print(f"values: {list(loop_range)}")
    results = []
    args = [(path, url, cursor, section, json_keys, col_names, step, disable) for cursor in loop_range]
    #print(f'args: {len(args)}\n{args}')
    tq.thread_map(lambda p: get_big_data(*p), args, desc='get_big_data', total=len(args))
 
def get_big_data(sc:SparkContext, path:str, url:str, cursor:int, json_keys:List[str], col_names:List[str], step:int, disable:bool = True):
    result_list = [get_json_data(f'{url}/{cursor}')]
    df = query_to_df(sc, result_list, section, json_keys, col_names, range(cursor, cursor + step, step), disable)
    df.write.option("header", true).partitionBy("DOI").parquest(path=getFileName(getPath()))
    #time.sleep(0.001)  # to visualize the progress

def query_to_df(results:pyspark.RDD, section: str, keys:List[str], col_names:List[str], loop_list:range, disable:bool = True) -> pyspark.sql.DataFrame:
    args = zip(results,
               section,
               [keys for _ in loop_list],
               loop_list,
               [disable for _ in loop_list])
    args = list(args)
    total = len(args)
    result_list = []
    result_list = tq.thread_map(lambda p: process_data(*p), args, desc=f'processing data {current_thread().name}', total=total, leave=False, disable=disable)
    
    data = np.array(flatten(result_list))
    #print(data)
    #print(col_names)0
    #print(f'loop_list: {loop_list}')
    return create_df(data, col_names)

def process_doi_data(path:str, url:str, doi:pd.Series, meta: List, col_names: List, item:int, disable:bool = False):
    results = get_data(url, doi[item:item+step], disable)
    tmp = list(results)
    df = query_to_df([r for r in tmp], meta, col_names, 
                     range(item, item + (len(tmp) * step), step), disable)
    df.to_parquet(pathlib.Path(f'{path}/{datetime.utcnow().timestamp()}.parquet'))

freq_count = lambda x,y: x[y].value_counts()

flatten = lambda y: sorted([sublist for inner in y for sublist in inner],
                           key=lambda x:x[0])

create_df = lambda x, y: x.toDF(y)

# Global Variables

In [2]:
print(f'Slurm id: {arc_pyspark.getSlurmID()}')
print(f'Does {arc_pyspark.getSlurmDir()} exist: {arc_pyspark.getSlurmDir().exists()}')
file.setBasePathName(str(arc_pyspark.getSlurmDir()))
pyfiles = f'{os.path.expanduser("~")}/proj/review/Peer-Review-22-23/utils.zip'
#pyfiles = 'utils.zip'
print(f'pyfiles:\n{pyfiles}')
sc, sqlCtx = arc_pyspark.getSparkSqlContext(pyfiles)
#print(sys.path)
print(sc, sqlCtx)

Slurm id: 17882055
Does /scratch/17882055 exist: True
pyfiles:
/home/albert.leung/proj/review/Peer-Review-22-23/utils.zip
configs:
[('spark.submit.pyFiles', '/home/albert.leung/proj/review/Peer-Review-22-23/utils.zip'), ('spark.ui.enabled', 'true'), ('spark.master', 'spark://mc50:33369'), ('spark.authenticate.secret', 'HFU0t72c'), ('spark.app.name', 'pyspark-shell'), ('spark.ui.killEnabled', 'false'), ('spark.files', '/home/albert.leung/proj/review/Peer-Review-22-23/utils.zip'), ('spark.executor.memory', '32G'), ('spark.default.parallelism', '16'), ('spark.speculation', 'false'), ('spark.driver.maxResultSize', '0'), ('spark.submit.deployMode', 'client'), ('spark.driver.memory', '32G'), ('spark.ui.showConsoleProgress', 'true'), ('spark.authenticate', 'true')]
spark ui: http://mc50:4040
<SparkContext master=spark://mc50:33369 appName=pyspark-shell> <pyspark.sql.session.SparkSession object at 0x7ff12a66e1c0>


## Case \#1: Pre-publish Data query for given timeframe

In [3]:
sc.getConf().getAll()

[('spark.ui.enabled', 'true'),
 ('spark.master', 'spark://mc50:33369'),
 ('spark.authenticate.secret', 'HFU0t72c'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir',
  'file:/home/albert.leung/proj/review/Peer-Review-22-23/spark-warehouse'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.ui.killEnabled', 'false'),
 ('spark.driver.port', '46715'),
 ('spark.driver.host', 'mc50'),
 ('spark.executor.memory', '32G'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.default.parallelism', '16'),
 ('spark.app.id', 'app-20230210010954-0005'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.maxResultSize', '0'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.startTime', '1676016593506'),
 ('spark.driver.memory', '32G'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.authenticate', 'true')]

In [4]:
# "global" variables
base_url: str = 'https://api.biorxiv.org'
query_type: str = 'details'
server: str = 'biorxiv'
start_interval: str = '2011-01-01'
end_interval: str = '2020-12-31'
step = 100
#print(get_total(f'{base_url}/{query_type}/{server}/{start_interval}/{end_interval}'))

### BioRxiv Server

In [5]:
#result = get_json_data(0,"https://api.biorxiv.org/details/biorxiv/2020-08-21/2020-08-28")
url: str = f'{base_url}/{query_type}/{server}/{start_interval}/{end_interval}'
file.setPath(f'data/prepub-{query_type}-{server}-{start_interval}!{end_interval}-{datetime.now()}')
step = 100

#finding the number of records for a given time period
#tmp = f"{base_url}/{start_interval}/{end_interval}"
#tmp = 'https://api.biorxiv.org/details/biorxiv/2020-08-21/2020-08-28'

#t = udf(lambda y:y, prepub_schema)
#df = sqlCtx.read.schema(prepub_schema).json(result)
#df.show()

'''
prepub_schema = StructType([
    StructField("messages", ArrayType(
        StructType([
#            StructField("status", StringType(), False),
#            StructField("interval", StringType(), False),
            StructField("cursor", IntegerType(), False),
#            StructField("count", IntegerType(), False),
#            StructField("count_new_papers", IntegerType(), False),
#                                                               StructField("total", IntegerType(), False)
        ])
    )),
    StructField("collection", ArrayType(
        StructType([
            StructField("doi", StringType()),
            StructField("title", StringType()),
            StructField("author", StringType()),
            StructField("author_corresponding", StringType()),
            StructField("author_corresponding_institution", StringType()),
            StructField("date", StringType()),
            StructField("version", StringType()),
            StructField("type", StringType()),
#            StructField("license", StringType(), False),
            StructField("category", StringType()),
#            StructField("jatsxml", StringType(), False),
#            StructField("abstract", StringType(), False),
            StructField("published", StringType()),
#            StructField("server", StringType(), False)
        ])
    ))
])                     
print(type(prepub_schema))
'''

prepub_schema = StructType([
    StructField("index", IntegerType()),
    StructField("doi", StringType()),
    StructField("title", StringType()),
    StructField("author", StringType()),
    StructField("author_corresponding", StringType()),
    StructField("author_corresponding_institution", StringType()),
    StructField("date", StringType()),
    StructField("version", StringType()),
    StructField("type", StringType()),
#            StructField("license", StringType(), False),
    StructField("category", StringType()),
    StructField("jatsxml", StringType()),
#            StructField("abstract", StringType(), False),
    StructField("published", StringType()),
#            StructField("server", StringType(), False)
])

spark_rdd_querying_processing(sc, sqlCtx, file.getPath(file.getBasePathName()), url, prepub_schema, "collection",
                              ["doi", "title", "authors", "author_corresponding", "author_corresponding_institution", "date", "version", "type", "category", "jatsxml", "published"],
                              ["index", "DOI", "Title", "Authors", "Corresponding_Authors", "Institution", "Date", "Version", "Type", "Category", "Xml", "Published"],
                              range (0, get_total(url), step))

#["doi", "title", "authors", "author_corresponding", "author_corresponding_institution", "date", "version", "type", "category", "published"],
    
#print(f'results:\n {results.take(2)}')
#results = args.flatMap(lambda p: get_json_data(*p))
#print(f'flatten:\n{results.take(5)}')

1: /scratch/17882055/data/prepub-details-biorxiv-2011-01-01!2020-12-31-2023-02-10 01:09:58.666178/


filename: /scratch/17882055/data/prepub-details-biorxiv-2011-01-01!2020-12-31-2023-02-10 01:09:58.666178/1676041801.877498.parquet


In [10]:
print(getBasePathName())
print(getPath())
print(getFileName(getPath()))
#journal_df = sqlCtx.read.parquet(getFileName(getPath()))
#journal_df.show()
journal_df = pd.read_parquet(getFileName(getPath()))
journal_df = create_prepublish_df(journal_df)
display(journal_df)


None
/scratch/17864919/data/prepub-details-biorxiv-2011-01-01!2020-12-31-2023-02-08 01:08:28.522038/
/scratch/17864919/data/prepub-details-biorxiv-2011-01-01!2020-12-31-2023-02-08 01:08:28.522038/1675868800.961023.parquet


AnalysisException: Path does not exist: file:/scratch/17864919/data/prepub-details-biorxiv-2011-01-01!2020-12-31-2023-02-08 01:08:28.522038/1675868800.961023.parquet

#### Cleaning and fixing missing data

In [7]:
import re
#pd.options.mode.use_inf_as_na = True # this option check for empty strings as well

jl_fix_df = pd.DataFrame(columns=journal_df.columns)
for col in journal_df:
    jl_fix_df = pd.concat([jl_fix_df, journal_df[journal_df[col].isna()]])
    jl_fix_df = pd.concat([jl_fix_df, journal_df[journal_df[col] == '']])

pattern = re.compile('^-|$-')
#display(journal_df[journal_df.Authors.str.contains(pattern)])
jl_fix_df = pd.concat([jl_fix_df, journal_df[journal_df.Authors.str.contains(pattern)]])
jl_fix_df.to_csv('data/data_fixing.csv')
display(jl_fix_df)

/global/software/spark/spark-3.2.0-bin-hadoop3.2/python/pyspark/pandas/internal.py:1536: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
23/02/07 21:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/07 21:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/07 21:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/07 21:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/07 21:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause seriou

,index,DOI,Title,Authors,Corresponding_Authors,Institution,Date,Version,Type,Category,Xml,Published,Num_of_Authors
508,139401,10.1101/2020.10.14.340273,Impulsivity and Thought Suppression in Behavio...,"Wan, L.; Zha, R.; Ren, J.; Li, Y.; Zhao, Q.; Z...",,University of Science and Technology of China,2020-10-15,1,new results,neuroscience,https://www.biorxiv.org/content/early/2020/10/...,NA,7
12033,116687,10.1101/2020.05.06.071340,A Novel Regulatory Role for the Circadian Cloc...,"Kay, S. A.; LI, Y.; CHANG, Y.; Nagel, D.; Liwa...",,University of Southern California,2020-05-08,1,new results,plant biology,https://www.biorxiv.org/content/early/2020/05/...,NA,5
16975,94799,10.1101/843763,STRESS-INDUCED GENETIC CHANGE IN FLAX REVEALS ...,"Li, X.",,case western reserve university,2019-11-15,1,new results,genomics,https://www.biorxiv.org/content/early/2019/11/...,NA,1
21671,109628,10.1101/2020.03.18.995985,Direct Measurement of Trunk Volume in Forest T...,"Leverett, R. T.; Ruskin, D. N.; Masino, S. A.",,Native Tree Society; Friends of Mohawk Trail S...,2020-03-20,1,new results,ecology,https://www.biorxiv.org/content/early/2020/03/...,NA,3
24029,80783,10.1101/708800,CLEC-2 suppresses calcification in cultured os...,"Kanai, T.; Sawa, Y.; Takara, K.; Kajiwara, K.;...",,Okayama University Graduate School of Medicine...,2019-07-19,1,new results,cell biology,https://www.biorxiv.org/content/early/2019/07/...,NA,8
72452,133701,10.1101/2020.08.30.266148,Imaging transmembrane dynamics of biomolecules...,"Hou, W.; Ma, D.; He, X.; Han, W.; Ma, J.; Xu, ...",,"Institute of Physics, Chinese Academy of Sciences",2020-08-31,1,new results,biophysics,https://www.biorxiv.org/content/early/2020/08/...,10.1021/acs.nanolett.0c03941,12
72896,101419,10.1101/2020.01.16.908798,Pterostilbene Protects Cochlea from Ototoxicit...,"ÖZDAS, S.; TASTEKIN, B.; GÜRGEN, S. G.; ÖZD...",,"Hacettepe University, Faculty of Science",2020-01-16,1,new results,neuroscience,https://www.biorxiv.org/content/early/2020/01/...,10.1371/journal.pone.0228429,8
134248,12673,10.1101/104778,Engaging narratives evoke similar neural activ...,"Cohen, S. S.; Henin, S.; Parra, L. C.",,The City College of the City University of New...,2017-01-31,1,new results,neuroscience,https://www.biorxiv.org/content/early/2017/01/...,10.1038/s41598-017-04402-4,3
139826,12153,10.1101/092171,Darwin: A Hardware-acceleration Framework for ...,"Turakhia, Y.; Zheng, K. J.; Bejerano, G.; Dall...",,Stanford University,2017-01-15,1,new results,genomics,https://www.biorxiv.org/content/early/2017/01/...,10.1109/MM.2019.2910009,4
139827,12154,10.1101/092171,Darwin: A Hardware-acceleration Framework for ...,"Turakhia, Y.; Zheng, K. J.; Bejerano, G.; Dall...",,Stanford University,2017-01-24,2,new results,genomics,https://www.biorxiv.org/content/early/2017/01/...,10.1109/MM.2019.2910009,4


In [ ]:
#fixed_df = pd.read_csv('')
#journal_df.merge(fixed_df, 'left', left_index=True)

In [ ]:
display(journal_df[journal_df.Published =='NA'])

### Publication data for Prepublication data for the given time period

In [ ]:
#from tqdm import tqdm_notebook, tnrange
from tqdm.notebook import tqdm_notebook
import time
import math

#variables
#https://api.biorxiv.org/pubs/biorvix/10.1101/759530
base_url: str = 'https://api.biorxiv.org'
query_type: str = 'pubs'
server: str ='biorxiv'
url: str = f'{base_url}/{query_type}/{server}'
path: str = f'pub_journal-{query_type}-{server}-{datetime.now()}'
step = 100
#doi = ['10.1101/856302']
doi = journal_df[journal_df.Published != 'NA'].Published

#create directory
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
total = len(doi)
#def process_doi_data(path:str, url:str, doi:pd.Series, meta: List, col_names: List, item:int, loop_list, disable:bool = False):
args = [(path, url, doi, 
         ["preprint_doi", "published_doi", "preprint_title", "preprint_authors", "preprint_author_corresponding", "preprint_author_corresponding_institution", "preprint_category", "published_journal", "preprint_date", "published_date"],
         ["DOI", "pub_DOI", "Title", "Authors", "Corresponding_Authors", "Institution", "Category", "Journal", "Preprint_Date", "Published_Date"],
         item, True) for item in range(0, total, step)]
print(f'total doi: {total} iter: {total/step}')
#tq.thread_map(lambda p: process_doi_data(*p), args, desc='process_doi_data', max_workers=5, total=len(args))
result = tq.thread_map(lambda p: process_doi_data(*p), args, desc='process_doi_data', total=len(args))


In [ ]:
filtered_df = pd.read_parquet(pathlib.Path(path))
print([item for item in filtered_df.Published_Date if len(item.strip()) != 10])
filtered_df = create_published_df(filtered_df).reindex()
filtered_df.rename(columns={'pub_DOI':'Published', 'Preprint_Date':'Date'}, inplace=True)


Finding and cleaning missing data

In [ ]:
pd.options.mode.use_inf_as_na = True # this option check for empty strings as well
fix_df = pd.DataFrame(columns=filtered_df.columns)
for col in filtered_df:
    fix_df = pd.concat([fix_df, filtered_df[filtered_df[col].isna()]])
    fix_df = pd.concat([fix_df, filtered_df[filtered_df[col] == '']])
    
    
display("Rows with incomplete data:", fix_df)

In [ ]:
print(f'\nPrepublished Artices w/ Publication Info that have missing info: {filtered_df.isnull().sum()}\n')
#print('Published data:\n',[(name, journal_df[name].isnull().sum()) for name in journal_df.columns if journal_df[name].isnull().values.any()])
#print('Prepublish data:\n',[(name, filtered_df[name].isnull().sum()) for name in filtered_df.columns if filtered_df[name].isnull().values.any()])
fix_df = filtered_df[filtered_df['Published_Date'].isna()]
display("Rows with NaN:", fix_df)

In [ ]:
filtered_df.info()
clean_df = filtered_df.dropna()
merged_df = pd.merge(journal_df, clean_df, how='right', on=['DOI','Published','Title', 'Authors','Corresponding_Authors','Institution','Category','Date','Num_of_Authors'])

In [ ]:
pd.options.mode.use_inf_as_na = True
print(f'Prepublished Artices w/ Publication Info: {merged_df.isnull().sum()}')
display(merged_df)

#### General Descriptive Anaylsis

In [ ]:
#summary of table
journal_df.describe(include='all', datetime_is_numeric=True)


In [ ]:
# published vs unpublished
def percentage (item1, item2) -> float:
    return round(item1 / (item1 + item2) * 100, 2)

journal_df['NotPublished'] = journal_df['Published'] == 'NA'
tmp_df = journal_df.groupby('DOI', as_index=False).first()
display(tmp_df)
result = freq_count(tmp_df,'NotPublished')

print(f'published: {result.iloc[0]} ({percentage(result.iloc[0],result.iloc[1])}%)')
print(f'not published: {result.iloc[1]} ({percentage(result.iloc[1],result.iloc[0])}%)')      

In [ ]:
#freq count of Instituion
freq_count(journal_df, 'Institution')

In [ ]:
#freq count of number of version
freq_count(journal_df, 'Version')

In [ ]:
#freq count of Type
freq_count(tmp_df, 'Type')

In [ ]:
#freq count Category
freq_count(tmp_df, 'Category')

#### Check

In [ ]:
assert get_total(url) == journal_df.shape[0]
"Total number of papers submitted and the length of the dataframe match."

The above code runs through all papers that are submitted within selected time frame. Error discussed during the previous meeting has been fixed. 

----

## Case 2: Published Articles query for a given timefame

In [ ]:
# "global" variables
#https://api.biorxiv.org/pubs/biorvix/2018-08-21/2018-08-28
base_url: str = 'https://api.biorxiv.org'
query_type: str = 'pubs'
server: str = 'biorxiv'
start_interval: str = '2011-01-01'
end_interval: str = '2020-12-31'


### BioRvix Server

In [ ]:
# "local" variables
url: str = f'{base_url}/{query_type}/{server}/{start_interval}/{end_interval}'
path: str = f'data/pub-{query_type}-{server}-{start_interval}!{end_interval}-{datetime.now()}'
step = 100
pubs_df = None

#finding the number of records for a given time period

pathlib.Path(path).mkdir(parents=True, exist_ok=True)
print(url)

multithread_processor(path, url, 
                      ["preprint_doi", "published_doi", "preprint_title", "preprint_authors", "preprint_author_corresponding", "preprint_author_corresponding_institution", "preprint_category", "published_journal", "preprint_date", "published_date"],
                      ["DOI", "pub_DOI", "Title", "Authors", "Corresponding_Authors", "Institution", "Category", "Journal", "Preprint_Date", "Published_Date"],
                      step, range(0, get_total(url), 100), True)


In [ ]:
pubs_df = pd.read_parquet(pathlib.Path(path)).sort_index(kind='mergesort', key=lambda x: x.astype(int))

In [ ]:
pubs_df.head(1)
#print([(index, value, datetime.strptime(value.split(':')[0], '%Y-%m-%d').date()) for index, value in enumerate(pubs_df.Published_Date) if len(value.strip()) != 10])
print([(index, pubs_df.loc(index, 'DOI'), value) for index, value in enumerate(pubs_df.Published_Date) if len(value.strip()) != 10])



In [ ]:

pubs_df = create_published_df(pubs_df)

#display(pubs_df)
#pubs_df.shape


In [ ]:
pubs_df.info()
print(pubs_df.Published_Date.isna().values.any(), pubs_df.Preprint_Date.isna().values.any())
print(pubs_df.Published_Date.subtract(pubs_df.Preprint_Date).where(pubs_df.Published_Date >= pubs_df.Preprint_Date))
#for index in range(pubs_df.shape[0]):
#    try:
#        pubs_df.loc(index, 'Published_Date') - pubs_df.loc(index, 'Preprint_Date')
#    except Exception as e:
        #print(e)
        #print (f'index: {index} Published_Date: {')

#### General Descriptive Analysis

In [ ]:
pubs_df.info()

In [ ]:
#summary of table
pubs_df.describe(include='all', datetime_is_numeric=True)


In [ ]:
#freq count of Num_of_Authors
freq_count(pubs_df,'Num_of_Authors')

In [ ]:
#freq count "Corresponding_Authors"
freq_count(pubs_df, 'Corresponding_Authors')

In [ ]:
#freq_count Institution
freq_count(pubs_df, 'Institution')

In [ ]:
#freq_count Category
freq_count(pubs_df, 'Category')

In [ ]:
#freq count Jounral
freq_count(pubs_df, 'Journal')

In [ ]:
#freq count Preprint_Date
freq_count(pubs_df, 'Preprint_Date')

In [ ]:
#freq count Published_date
freq_count(pubs_df, 'Published_Date')

In [ ]:
#freq count by category
tmp = pubs_df
tmp['Time_Month'] = pubs_df.Published_Date.subtract(pubs_df.Preprint_Date).divide(np.timedelta64(1, 'M')).where(pubs_df.Published_Date > pubs_df.Preprint_Date)
#tmp.Time_Month = tmp.
tmp.groupby(['Category'], as_index=True).Time_Month.describe()

In [ ]:
#Time duration by Category - Journal
tmp.groupby(['Category','Journal'], as_index=True).Time_Month.describe().to_csv('data/Category-Journal_duration.csv')

In [ ]:
#Time duration by Journal - Category
tmp.groupby(['Journal','Category'], as_index=True).Time_Month.describe().to_csv('data/Journal-Category_duration.csv')

#### Check

In [ ]:
'''
                      columns=["pre_DOI", "pub_DOI",
                               "Title", "Authors", "Corresponding_Authors",
                               "Institution",
                               "Category", "Journal", "Preprint_Date", "Published_Date"])
'''
#assert get_total(f'{base_url}{query_type}{server}{start_interval}{end_interval}') == pubs_df.shape[0]
"Total number of papers published and the length of the dataframe does match."

----

### Detailed analysis of published articles for a given timeframe

In [ ]:
from multipledispatch import dispatch
@dispatch(pd.DataFrame, int, str)
def get_values(df: pd.DataFrame, row: int, col: str):
    return get_values(df, [row], [col])

@dispatch(pd.DataFrame, int, list)
def get_values(df: pd.DataFrame, row: int, cols: List[str]):
    return get_values(df, [row], cols)

@dispatch(pd.DataFrame, list, str)
def get_values(df: pd.DataFrame, rows: List[int], col: str):
    return get_values(df, rows, [col])

@dispatch(pd.DataFrame, list, list)
def get_values(df: pd.DataFrame, rows: List[int], cols: List[str]) -> List[List[str]]:
    return [[df.at(row, col) for col in cols] for row in rows]

@dispatch(pd.DataFrame, str)
def get_values(df: pd.DataFrame, col: str):
    return df[col]

@dispatch(pd.DataFrame, list)
def get_values(df: pd.DataFrame, cols: List[str]):
    return df[cols]

                                                                                                                                                                          ### BioRvix Service

In [ ]:
#from tqdm import tqdm_notebook, tnrange
from tqdm.notebook import tqdm_notebook
import time
import math

#variables
#https://api.biorxiv.org/pubs/biorvix/2018-08-21/2018-08-28
#https://api.biorxiv.org/details/biorxiv/10.1101/759530
#https://api.biorxiv.org/details/biorvix/10.1101/759530
base_url: str = 'https://api.biorxiv.org'
query_type: str = 'details'
server: str ='biorxiv'
url: str = f'{base_url}/{query_type}/{server}'
path: str = f'data/prepub-{query_type}-{server}-{datetime.now()}'
step = 100
#doi = ['10.1101/856302']
doi = pubs_df.DOI
prepub_df = None

'''
for item in tqdm_notebook(range(0, total, step), desc='get_prepublish_data', 
                 total=math.ceil(total/step)):
    #time.sleep(0.005)
    results = get_data(None, 
                       url, 
                       doi[item:item+step])
    tmp = list(results)
    prepub_df = query_to_df([r for r in tmp], 
                            ["doi", "title", "authors", "author_corresponding", "author_corresponding_institution", "date", "version", "type", "category", "published"],
                            range(item, item + (len(tmp) * step), step),
#                            range(item, (item + ((len(tmp[counter]) - 1) * step)) * 10, step),
                            ["DOI", "Title", "Authors", "Corresponding_Authors", "Institution", "Date", "Version", "Type", "Category", "pub_DOI"], False)
    #prepub_df = create_prepublish_df(prepub_df)
    prepub_df.to_parquet(pathlib.Path(f'{path}/{datetime.utcnow().timestamp()}.parquet'))
'''

#create directory
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
total = len(doi)
args = [(path, url, doi, item, step, True) for item in range(0, total, step)]
#print(f'args: {len(args)}\n{args}')
print(url, f'total doi: {total} iter: {total/step}')
#tq.thread_map(lambda p: process_doi_data(*p), args, desc='process_doi_data', max_workers=5, total=len(args))
tq.thread_map(lambda p: process_doi_data(*p), args, desc='process_doi_data', total=len(args))


In [ ]:
prepub_df = create_prepublish_df(pd.read_parquet(pathlib.Path(path)))

prepub_df

In [ ]:
print('Published data:\n',[(name, pubs_df[name].isnull().sum()) for name in pubs_df.columns if pubs_df[name].isnull().values.any()])
print('Prepublish data:\n',[(name, prepub_df[name].isnull().sum()) for name in prepub_df.columns if prepub_df[name].isnull().values.any()])

In [ ]:
combined_df = pd.merge(pubs_df,prepub_df, how="outer", on=['DOI','Title', 'Authors', 'Corresponding_Authors', 'Institution', 'Category', 'Num_of_Authors', 'pub_DOI'])
#combined_df = pd.merge(pubs_df,prepub_df, how="right", on=['DOI','Title', 'Authors', 'Corresponding_Authors', 'Institution', 'Category', 'Num_of_Authors'])
combined_df['Time_month'] = (combined_df['Published_Date'] - combined_df['Preprint_Date']) / np.timedelta64(1, 'M')
combined_df['Keep'] = combined_df.Published_Date > combined_df.Preprint_Date
#ref = combined_df.set_index(['pub_DOI'], inplace=False)
combined_df.head(1)
#display(combined_df.groupby('DOI').count())

In [ ]:
print([(index, value) for index, value in enumerate(combined_df.groupby(['DOI'])['Time_month']) ])


In [ ]:
# checking for NA
print([(name, combined_df[name].isnull().sum()) for name in combined_df.columns if combined_df[name].isnull().values.any()])
print([(name, combined_df.groupby([name], dropna=False).size) for name in combined_df.columns if (combined_df.groupby([name], dropna=False)[name].count() > 0).any()])
print([(name, combined_df.groupby([name], dropna=True).size) for name in combined_df.columns if (combined_df.groupby([name], dropna=False)[name].count() > 0).any()])

In [ ]:
#%%capture cap
display(prepub_df.shape, pubs_df.shape, combined_df.shape, combined_df.groupby(['pub_DOI'], as_index=True).count().shape)
#test = combined_df.groupby(['DOI', 'pub_DOI', 'Category', 'Institution', 'Corresponding_Authors', 'Journal', 'Preprint_Date', 'Published_Date', 'Authors', 'Num_of_Authors', 'Title', 'Date', 'Version'])
test = combined_df.groupby(['DOI'], as_index = True)
#display(test.iloc[test.Title.nunique().ne(1),:])
#display(test.Title.nunique().ne(1).index)
#display(test.Title.nunique().ne(1))
#key = test.Title.nunique().ne(1)
#display(ref.index.isin(key))
#display(ref[ref.index.isin(key)])
#display(test.Title.nunique().ne(1).shape)
#test.Title.indices.
#display(test.Title.indices)
#df.at only get signal value so need to use list comprehension to get multiple columns in a particular row
#print(test.Title.indices['10.1128/IAI.00353-19'])
#display(combined_df .iloc[[1826, 1827]])
#display(combined_df)
combined_df.groupby(['pub_DOI'], as_index=False).Title.unique()

In [ ]:
combined_df.groupby(['Category', 'Keep'], as_index=True).Time_month.describe()

**NB:** There are some issues with the above table since there are 29 publications that have more than one enteries.

In [ ]:
df = combined_df.loc[combined_df.reset_index().groupby(['pub_DOI'])['Time_month'].idxmax()]


In [ ]:
display(df.groupby(['Category']).Time_month.describe())

In [ ]:
combined_df.groupby(['Institution'], as_index=True).pub_DOI.describe().to_csv("data/general.csv")

In [ ]:
combined_df.groupby(['Institution', 'Category'], as_index=True).Time_month.describe()

In [ ]:
reduce_df = combined_df[combined_df.Keep == True]
reduce_df.groupby(['Category', 'Keep'], as_index=True).Time_month.describe()

In [ ]:
print(pubs_df.shape)
print(prepub_df.shape)
print(test.count().shape)
print((test.Title.nunique() - 1 ).shape)

In [ ]:
test.filter(lambda x: x['Title'].nunique() > 1).to_csv('data/title-change.csv')
prepub_df.to_csv('data/pre&pub-data.csv')
pubs_df.to_csv('data/pub_data.csv')

# Junk Code
Please disregard the codes below.

In [14]:
R = sc.parallelize([3, 4, 5])
output = R.map(lambda x: [[x, y] for y in range(1, x)]).collect()
print("Output : ",output)
 
S = sc.parallelize([3, 4, 5])
output1 = S.flatMap(lambda x: [[x, y] for y in range(1, x)]).collect()
print("Output : ",output1)

Output :  [[[3, 1], [3, 2]], [[4, 1], [4, 2], [4, 3]], [[5, 1], [5, 2], [5, 3], [5, 4]]]
Output :  [[3, 1], [3, 2], [4, 1], [4, 2], [4, 3], [5, 1], [5, 2], [5, 3], [5, 4]]


In [ ]:
'''
for item in tqdm_notebook(range(0, total, step), desc='get_prepublish_data', 
                 total=math.ceil(total/step)):
    #time.sleep(0.005)
    results = get_data(None, 
                       url, 
                       doi[item:item+step])
    tmp = list(results)
    prepub_df = query_to_df([r for r in tmp], 
                            ["doi", "title", "authors", "author_corresponding", "author_corresponding_institution", "date", "version", "type", "category", "published"],
                            range(item, item + (len(tmp) * step), step),
#                            range(item, (item + ((len(tmp[counter]) - 1) * step)) * 10, step),
                            ["DOI", "Title", "Authors", "Corresponding_Authors", "Institution", "Date", "Version", "Type", "Category", "pub_DOI"], False)
    #prepub_df = create_prepublish_df(prepub_df)
    prepub_df.to_parquet(pathlib.Path(f'{path}/{datetime.utcnow().timestamp()}.parquet'))
'''



In [ ]:
# "local" variables
url: str = f'{base_url}/{query_type}/{server}/{start_interval}/{end_interval}'
path: str = f'pub-{query_type}-{server}-{start_interval}!{end_interval}-{datetime.datetime.now()}'
step = 100
pubs_df = None

#finding the number of records for a given time period

pathlib.Path(path).mkdir(parents=True, exist_ok=True)
print(url)
results = get_data(None, 
         url, 
         range(0, get_total(url), 100))

tmp = list(results)
for item in range (0, get_total(url), step):
    result_list = tmp[item:(item+step)]
    pubs_df = query_to_df(result_list, 
                          ["preprint_doi", "published_doi", "preprint_title", "preprint_authors", "preprint_author_corresponding", "preprint_author_corresponding_institution", "preprint_category", "published_journal", "preprint_date", "published_date"],
                          range(item, len(result_list) * step, step),
                          ["DOI", "pub_DOI", "Title", "Authors", "Corresponding_Authors", "Institution", "Category", "Journal", "Preprint_Date", "Published_Date"])
    pubs_df.to_parquet(pathlib.Path(f'{path}/{datetime.datetime.utcnow().timestamp()}.parquet'))
pubs_df = pd.read_parquet(pathlib.Path(path))
pubs_df = create_published_df(pubs_df)

display(pubs_df)

In [ ]:
def get_publisher_prefix(doi: pd.Series) -> pd.Series:
    return doi.apply(lambda x: x.split("/")[0])

query_type='/publisher'

def get_publisher_data(url)
get_data(None,f'{base_url}{query_type}'
         get_publisher_prefix(pubs_df.pub_DOI).unique())

**Journal API**<br>
https://www.nature.com/opensearch/<br>
https://www.biorxiv.org/content/10.1101/339747v4<br>
https://www.biorxiv.org/content/10.1101/339747v4.full.pdf<br>
https://api.biorxiv.org/details/biorxiv/10.1101/099697


In [ ]:
journal_list = []

# `license`,`abstract`, and `server` are excluded from the metrics. 
for journal in json_info["collection"]:
    journal_list.append([journal["doi"], journal["title"], journal["authors"],
                         journal["author_corresponding"], 
                         journal["author_corresponding_institution"],
                         journal["date"], journal["version"], journal["type"],
                         journal["category"], journal["jatsxml"], journal["published"]])
    

In [ ]:
journal_df = pd.DataFrame(data=journal_list,
                         columns=["DOI", "Title", "Authors", "Corresponding Authors",
                                  "Institution",
                                  "Date", "Version", "Type", "Category", "Xml", "Published"])
journal_df.head()